<center>
<img style="float: center;" src="images/CI_horizontal.png" width="400">
</center>
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

<center> Julia Lane, Clayton Hunter, Brian Kim, Benjamin Feder, Ekaterina Levitskaya, Tian Lou, Lisa Osorio-Copete. 
</center>

# Dataset Exploration

## Table of Contents

JupyterLab contains a dynamic table of contents that can be accessed by clicking the sixth (second from bottom) icon on the left-hand toolbar.

## Introduction

In an ideal world, we have all of the data we want with all of the desirable properties (no missing values, no errors, standard formats, and so on). We'd also have perfect data documentation, with summary statistics and appropriate aggregate measures of everything we'd want to investigate. However, that is hardly ever true, and we have to use our datasets to answer questions of interest as intelligently as possible. 

In this notebook, we will discover the datasets we have on the ADRF and use them to answer some questions of interest.

### Learning Objectives

This notebook will give you the opportunity to spend some hands-on time with the data. We will base our discussions around the following questions:

- What does the TANF experience look like? 
- Where are TANF recipients finding employment? 

Throughout the notebook, you will be exposed to various programming techniques to help answer these questions. This notebook will form the basis of all the other types of analyses you will do in this class and is a crucial first step for any data analysis workflow. As you work through the notebook, we will have checkpoints for you try out your own code, but you can also think about how you might apply any of the techniques and code presented with other datasets as well. 

### Datasets We Will Explore In This Notebook
- **Indiana TANF data**: Data on TANF cases as well as those associated with TANF cases in Indiana.
- **Indiana Unemployment Insurance (UI) Wage data**: Indiana workers' quarterly earnings and employment. 

You will explore these datasets using a combination of SQL and R, as explained in the handy Leveraging R and SQL [document](leveraging_r_and_sql.md).

**This notebook will provide an introduction and examples for:**

- How to create new tables from the larger tables in database (sometimes called the "analytical frame")
- How to explore different variables of interest
- How to create aggregate metrics
- How to join tables
- How to generate descriptive statistics to describe a specific cohort

### Methods
With our SQL queries, we will:

- Learn about rows and columns in the data with basic queries using `SELECT`, `UNIQUE` and `ORDER BY` 
- Select subsets of tables from the database with `WHERE`
- Aggregate data over groups with `GROUP BY`

And we will use R to:

- Find measures broken down by group using `group_by` and `summarize`
- Sort values with `arrange` and `desc`
- Create new variables with `mutate`

### R Setup

Before we can use R functions in some specific packages that are not available in `base` R, we need to load them using the built-in function `library()`. For example, running `library(tidyverse)` loads the `tidyverse` suite of packages.

> When you run the following code cell, don't worry about the message below. You'll find out what it's saying later.

In [ ]:
#database interaction imports
library(DBI)
library(RPostgreSQL)

# data manipulation/visualization
library(tidyverse)

# scaling data
library(scales)

__When in doubt, use shift + tab to read the documentation of a method. Full documentation can be printed with `?<package/function_name>`, e.g. `?tidyverse/ggplot` or `?sprintf`.__ Do not worry about memorizing the information in the help documentation - you can always run this command when you are unsure of how to use a function.

> Certain functions exist across multiple packages (e.g. the function `lag` exists in both the `dplyr` and `stats` package - also noted in the message yielded from `library(tidyverse)`. When calling a function, you can put the package name first to ensure that you are using the right one. For example, `dplyr::lag` or `stats::lag` calls the `lag` function from `dplyr` or `stats`, respectively. 

In [ ]:
# see help documentation for sprintf
?sprintf

## Load the Data

Since we are working with the PostgreSQL database `appliedda` in this course, we will demonstrate how to use R to read data from a relational database.

### Establish a Connection to the Database

We will create the database connection using the `DBI`  and `RPostgreSQL` libraries. Each time you create a new notebook in this course, make sure you copy the following code chunk so you can connect to the database.

__Database Connection__

In [ ]:
# create an RPostgreSQL driver
drv <- dbDriver("PostgreSQL")

# connect to the database
con <- dbConnect(drv,dbname = "postgresql://stuffed.adrf.info/appliedda")

### Formulate Data Query

This part is similar to writing a SQL query in DBeaver. Depending on the data we are interested in, we can use different queries to pull different data. In this example, we will pull in 20 rows of person-level TANF spells data, which is stored in the `person_month` table inside the `in_fssa` schema.

__create a query as a `character` string object in R__

In [ ]:
# Create query character string
query <- "
SELECT *
FROM in_fssa.person_month
LIMIT 20;
"

Here, we use `LIMIT` to read in only the first 20 rows because we're just looking to preview the data and we don't want to eat up memory by reading a huge data frame into R. 

> `LIMIT` provides one simple way to get a "sample" of data; however, using `LIMIT` does **not provide a _random_** sample. You may get different samples of data than others using just the `LIMIT` clause, but it is just based on what is fastest for the database to return.

In [ ]:
# print with newlines instead of /n
writeLines(query)

### Reading in the Data 

Now we can use `con` and `query` as inputs to `dbGetQuery()` to read the data into an R Data frame. 

In [ ]:
# read in data frame and assign to df
df <- dbGetQuery(con,query)

In [ ]:
# see first few rows of df
head(df)

### What is in the Database?

We will start our exploration by looking at what is in the database. We will find the list of schema names in the database, the list of tables in these schemas, and the list of columns in these tables.


We will get this information from a built-in schema called `information_schema`, which provides metadata on the database. The `information_schema` schema has tables `tables`, `schemata`, and others, as you will see.

In general, as in DBeaver, database schemas and tables are denoted as `<schema_name>.<table_name>`. 

You will query the `information_schema` tables: 

- `information_schema.tables` for metadata on tables   
- `information_schema.schemata` for metadata on schemas   
- `information_schema.columns` for metadata on columns 

In [ ]:
# see 5 of the available schemas
query <- "
SELECT DISTINCT * 
FROM information_schema.schemata
limit 5;
"
dbGetQuery(con,query)

As a reminder, in this class you have access to read in data from the following schemas: `public`,`in_dwd`, `in_fssa`, and `ada_tdc_2020`. You only have write access to the `ada_tdc_2020` schema.

Let's see a few of the tables that exist in the `in_fssa` schema.

In [ ]:
# see five tables in the in_fssa schema
qry <- "
SELECT DISTINCT 
table_schema, table_name 
FROM information_schema.tables 
WHERE table_schema IN ('in_fssa')
limit 5
"

dbGetQuery(con,qry)

<font color=red><h3> Checkpoint 1: Explore Columns </h3></font> 

Take a look at the columns in the `person_month` table in the `in_fssa` schema. What are some of the variables that you might be able to use to answer the questions posed in the Learning Objectives?

> Refer to the data dictionary on the class website to understand what the different variables refer to.

In [ ]:
query <- "
SELECT * 
FROM information_schema.columns 
where table_schema = __ and table_name = __;
"

dbGetQuery(con,query)

### `sprintf`

We'll quickly cover the `sprintf` funtion, which we'll use to create our first flexible, or *parameterized*, query.

`sprintf` is a string manipulation function that enables us to use symbols as placeholders in R so we can interchange values in an expression. In this case, we would like to see the columns in the other schemas and tables provided to us. Rather than rewriting all the queries, we can use `sprintf` to parameterize the queries, making them much more flexible. 

`sprintf` takes the form:
`sprintf(base_string, input_parameter)`

The base string should have a placeholder to denote the part of the string that will be replaced by `input_parameter`.  

    `%s` - placeholder where the replacement is a string   
    `%d` - placeholder where the replacement is an integer

Let's take a simple example to see how `sprintf` works to see the variables in the `case_month` table in the `in_fssa` schema using our query from Checkpoint 1.

> We will leverage the power of `sprintf` in later notebooks, as it allows us to input values in R into SQL queries.

In [ ]:
# Column names in in_fssa.case_month

# assign schema and table here
schema <- "in_fssa"
tbl <- "case_month"

# use base query and assign table_schema and table_name to '%s'
base_query <- "SELECT * 
FROM information_schema.columns 
WHERE table_schema = '%s' AND table_name = '%s'
limit 5;"

# feed in the new schema and tbl to query
query <- sprintf(base_query,schema, tbl)

# see query
writeLines(query)

Then, we can just use the query the same way as usual. 

In [ ]:
# query from database
dbGetQuery(con, query)

## Defining our Cohort

In this section, you will begin to explore TANF individual spell data. As you work through this section, try to ask yourself questions such as: 
- What variables are you interested in? 
- What variables do you need to identify the sample you are interested in?
- In which table(s) are these variables available? 
- Are there any missing values in these variables?

We will focus on two specific cohorts: primary TANF benefit recipients who had a spell ending during 2016Q4 and those who spells ended during in 2009Q1. A spell is defined as a period in which an individual/household is receiving aid from TANF. 

The data we will use to define these two cohorts live in two tables in the `in_fssa` schema: `case_month` and `person_month`. As partially indicated by their names, `case_month` contains case-level information, such as start and end dates as well as the number of individuals pertaining to the case, and `person_month` provides individual-level information on every member of the case, such as a primary recipient indicator, some demographic variables, and a handful of case-level measures.

Let's start by looking at the `case_month` table. Here, we will try to identify variables we can use to define and subsequently understand our cohort.

In [ ]:
# see 5 entries of the case_month table
qry <- "
SELECT *
FROM in_fssa.case_month
LIMIT 5;
"

dbGetQuery(con, qry)

There are two columns in particular in `case_month` that we can use to help define our cohorts by their end dates: `rptmn` and `tanf_end`. These variables describe the reporting month/year of the case and if the month/year was the final month of the case, respectively.

Let's take a look at just the `rptmn` and `tanf_end` columns in `case_month`.

In [ ]:
# just look at rptmn and tanf_end
qry <- "
SELECT rptmn, tanf_end
FROM in_fssa.case_month
LIMIT 5;
"

dbGetQuery(con, qry)

However, we can't retrieve all of the information we need to create our cohort from just `case_month` since we cannot identify the primary recipients for each case. Let's see if we can identify primary recipients by using `person_month`.

In [ ]:
# see person_month
qry <- "
SELECT *
FROM in_fssa.person_month
LIMIT 5;
"

dbGetQuery(con, qry)

Can you spot the `affil` column? This will help us identify the `ssn` of the primary recipient for each case. From there, we can identify each TANF case using the common `caseid` column that exists across the two tables.

Also, it turns out that the same `rptmn` and `tanf_end` columns are available in `person_month`, so we can define our cohort just using the `person_month` table. However, when we want to understand the typical TANF experience within our cohort we will want to look at some of the case-level variables from `case_month`, so we will leverage both tables to define our two cohorts.

<font color=red><h3>Checkpoint 2: Isolate Primary Recipients</h3></font> 

Read 5 rows of the `person_month` table where the `ssn` corresponds to the primary recipient of the case into R. How did you figure out which value of `affil` is associated with the primary recipient?

In [ ]:
# find five primary recipients
query <- "

"

dbGetQuery(con,query)

### Create our cohorts

We have all the information we need to find our cohorts of primary recipients of TANF benefits who left the program sometime in 2009Q1 or 2016Q4. Let's start by creating our 2016Q4 table with the addition of variables tracking the length of the spell as well as total length of all spells as the primary recipient. In defining our cohort, we will need to specify a few points in our `WHERE` clause:

- `affil = '1'`, for primary recipients
- `tanf_end = true`, for concluding spells
- `substring(month,1,4) = '2016'`, for spells in 2016
- `substring(month,5,2) in ('10', '11', '12')`, for spells in October, November, or December

> We have also identified some `ssn` values as problematic, so we will not include them in our cohorts.

In [ ]:
# define 2016Q4 cohort
qry <- "
select ssn, caseid, tanf_start, tanf_end, tanf_spell_months, tanf_total_months, substring(month,1,4) as rep_year, 
substring(month,5,2) as rep_month, extract(year from dob) as dob_yr
from in_fssa.person_month
where affil = '1' and 
tanf_end=true and 
substring(month,1,4) = '2016' and 
substring(month,5,2) in ('10', '11', '12') and
ssn not in REDACTED
"
#read into R as df
df <- dbGetQuery(con,qry)

In [ ]:
#look at df
head(df)

Given your background knowledge of the TANF program, do you think that the same `ssn` could show up in `df` multiple times? Let's take a look because in this case, we do not want the same `ssn` to appear in multiple rows in our table.

In [ ]:
# see if any ssns appear more than once in df
df %>%
    count(ssn) %>%
    filter(n>1) %>%
    head()

As you can see above, there are some individuals who were the primary recipients of multiple TANF cases that ended in 2016Q4. Since we do not want repeating `ssn` values in understanding employment outcomes (otherwise overweighting these individuals' outcomes), let's take each `ssn`'s most recent exit within this time frame. We can find this by selecting distinct `ssn` values when ordering by exit month. Also, let's include one variable found only in the case-level data: the county of residence. This means that we need to join our cohort with the `case_month` table in the `in_fssa` schema.

> Since we only want `ssn`'s where the `caseid` also contains case-level information specific to the `case_month` table, we will use an `inner join`, as opposed to a `left join` or `right join`.

In [ ]:
# 2016Q4 cohort with most recent case information
qry <- "
SELECT distinct on (a.ssn)
a.ssn, a.caseid, a.month, a.tanf_start, a.tanf_end, a.tanf_spell_months, a.tanf_total_months,b.county,
substring(a.month,1,4) as rep_year, substring(a.month,5,2) as rep_month, extract(year from dob) as dob_yr
FROM in_fssa.person_month a
INNER JOIN in_fssa.case_month b 
on a.caseid = b.caseid
WHERE a.affil = '1' and
a.tanf_end = TRUE and 
ssn not in REDACTED and
substring(a.month,1,4) = '2016' and 
substring(a.month,5,2) in ('10','11','12')
order by a.ssn, a.month desc;
"

#read into R as df
df_2016 <- dbGetQuery(con,qry)

Just to make sure, let's confirm that there are not any `ssn` values that appear multiple times in `df_2016`. 

In [ ]:
# make sure no ssns appear in multiple rows in df_2016
df_2016 %>%
    count(ssn) %>%
    filter(n>1) %>%
    head()

Congratulations! You have successfully defined our 2016Q4 cohort.

<font color=red><h3> Checkpoint 3: Recreate for 2009Q1 </h3></font> 

Recreate `df_2016` for our cohort of primary recipients of TANF benefits that concluded during 2009Q1, including the same variables as `df_2016`, and save the cohort as `df_2009`.

In [ ]:
# Create cohort for 2009Q1
qry = "

"

df_2009 = dbGetQuery(con, qry)

## The TANF Experience

How can we leverage our data to better understand the TANF experience? We will make use of our two cohorts, `df_2009` and `df_2016`, to answer these questions:

1. How many individuals are in the 2016Q4 cohort?
1. What is the age breakdown of this cohort?
1. How does the number of individuals vary by county? 
1. What is the distribution of spell lengths within this cohort?
1. What are the spell lengths at the 10th, 25th, 50th, 75th and 90th percentiles?
1. Are we seeing a concentration of lengthy spells in specific regions?

We will walk through how to find these answers in R for our 2016Q4 cohort, and you will replicate the same analysis on `df_2009` in the checkpoint at the end of this section.

<font color=green><h4>Question 1: How many individuals are in the 2016Q4 cohort? </h4></font> 

In [ ]:
# because each row is a unique ssn, can just get the count of rows
nrow(df_2016)

<font color=green><h4>Question 2: What is the age breakdown of the cohort? </h4></font> 

Notice two columns in `df_2016`: `rep_year` and `dob_yr`. We can use these two columns to find the age breakdown of our cohort.

In [ ]:
# look at rep_year and dob_yr
df_2016 %>%
    select(rep_year, dob_yr) %>%
    head(5)

To find the ages of everyone in our cohort, we can subtract the `dob_yr` from `rep_year`. We can create this new column using `mutate()`.

In [ ]:
# see example of mutate
df_2016 %>%
    mutate(age = as.numeric(rep_year) - dob_yr) %>%
    select(rep_year, dob_yr, age) %>%
    head()

In [ ]:
# save to ages
ages <- df_2016 %>%
    mutate(age = as.numeric(rep_year) - dob_yr) %>%
    select(rep_year, dob_yr, age)

In [ ]:
head(ages)

To see age breakdowns of our cohort, let's bin the ages into a few groups. We can use `mutute()` with `case_when()` to accomplish this.

In [ ]:
# create age groups
ages <- ages %>%
    mutate(age_group = case_when(
        age < 18 ~ "0-17",
        between(age, 18, 39) ~ "18-39",
        between(age, 40, 59) ~ "40-59",
        age >= 60 ~ "60+"
    )
          )

In [ ]:
# see counts by age group
ages %>%
    count(age_group)

Does this age breakdown surprise you at all?

<font color=green><h4>Question 3: How does the number of individuals vary by county?</h4></font> 

First, let's look at a few rows of `df_2016` while focusing on the `ssn` and `county` values to get a sense of what they look like.

In [ ]:
head(df_2016)

In [ ]:
# look at ssn and county
df_2016 %>%
    select(ssn, county) %>%
    head()

It doesn't seem like there's anything crazy (incorrect data types, in particular), going on with either variable, so let's count the number of exiters by county for a few counties using the `count()` function from the tidyverse.

> Sometimes, you may come across some "craziness" (incorrect data types, weird values, etc.) in the data just by chance. The code above was just a quick check to make sure our `county` variable was of character type, as opposed to a numeric column, for instance.

In [ ]:
# count number of exiters by county
df_2016 %>%
    count(county) %>%
    head()

If you want to go a bit more in depth, you can sort by the counties with the most amount of individuals in our cohort using a combination of `arrange()` and `desc()`

In [ ]:
# top 10 most popular counties in our cohort
df_2016 %>%
    count(county) %>%
    arrange(desc(n)) %>%
    head(10)

Finally, if we want to match the county codes to the names of the counties, we can load the `tl_2016_us_county` table from the `public` schema into R and join the two data frames.

> Indiana's state fips code is 18.

In [ ]:
# Get county codes, county names, polygons, and centroids of those polygons for the Indiana state
qry <- 
"SELECT countyfp as county, name
FROM public.tl_2016_us_county
WHERE statefp = '18'
"
#read into R as df
counties <- dbGetQuery(con, qry)

In [ ]:
# see counties
head(counties)

Similar to SQL's `LEFT JOIN`, one of the `tidyverse` packages, `dplyr`, contains `left_join()` which we can use to match the county codes to their proper names.

In [ ]:
# save top 10 most popular counties in our cohort
top_cnty <- df_2016 %>%
    count(county) %>%
    arrange(desc(n)) %>%
    head(10)

# left join to county lookup table
    left_join(top_cnty, counties, by="county") %>%
    select(name, n)

Just from this summary you can get the sense that a decent chunk of our cohort received TANF benefits from a select few counties. In the [Data Visualization](04_01_Data_Visualization.ipynb) notebook, you will visualize the county breakdown of our cohorts using a heatmap.

<font color=green><h4>Guiding Question 4: What is the distribution of spell lengths within our cohort? </h4></font>

There are two simple options to find a basic numerical distribution:
1. Use base R's `summary()` function, not to be confused with the tidyverse's `summarize()`
1. Find specific percentiles using `quantile()`

We'll work through both of these techniques in Guiding Questions 3 and 4. Since we can answer this question with a generic summary, we will go with our first option here.

In [ ]:
# get quick summary of current spell lengths
summary(df_2016$tanf_spell_months)

<font color=green><h4>Guiding Question 5: What are the spell lengths at the 10th, 25th, 50th, 75th and 90th percentiles?</h4></font> 

Given this question's percentile requirements, we will use the `quantile()` function, which allows us to specify certain percentiles.

> For the purposes of your final project outputs, you will be asked to report "fuzzy" percentiles to maintain data confidentiality. This will be covered more in-depth in the [Disclosure Review](Disclosure_Review.ipynb) notebook.

In [ ]:
# use a list of percentile values for spell lengths
quantile(df_2016$tanf_spell_months, c(.1, .25, .5, .75, .9))

Upon first glance, you may find the output from `quantile()` to be a bit hard to read. Luckily, we can use `quantile` in conjunction with the tidyverse's `summarize()` function to output an easy-to-read data frame.

In [ ]:
# make it look a little prettier
df_2016 %>%
    summarize('.1' = quantile(tanf_spell_months, .1),
              '.25' = quantile(tanf_spell_months, .25),
              '.5' = quantile(tanf_spell_months, .5),
              '.75' = quantile(tanf_spell_months, .75),
              '.9' = quantile(tanf_spell_months, .9)
             )

Do any of these numbers suprise you? Or is this what you were expecting to see?

<font color=green><h4>Guiding Question 6: Are we seeing a concentration of lengthy spells in specific regions?</h4></font> 

Now, let's look at the counties that have a high proportion of recipients with long spells. For the purposes of this exercise, we will define a long-term stayer as a recipient with a spell longer than the 90th percentile value. Our process will be as follows:
- Identify the 90th percentile of the amount of months of these spells
- Create an indicator variable using `mutate()` for whether each spell was greater than that of the 90th percentile
- Find the proportion of lengthy spells by county

First, we will identify the value at the 90th percentile using our code above. This time, we will assign the output to a specific variable.

In [ ]:
# assign better looking 90th percentile output to percentile
percentile <- df_2016 %>%
    summarize('.9' = quantile(tanf_spell_months, .9))

percentile

Next, we will test some code to make sure we properly create our indicator variable `rel_length` for each spell in our data frame.

In [ ]:
# test if code will work to create long vs not long depending on 
# if spell length is greater than the 90th percentile of all spell lengths
# call indicator variable "rel_length"
df_2016 %>%
    select(ssn, county, tanf_spell_months) %>%
    mutate(rel_length = ifelse(tanf_spell_months > percentile$'.9', 'long', 'not long')) %>%
    head()

Given that it seems like we can create `rel_length` using this code, let's store `df_2016` with our new indicator variable as a new data frame `lens`.

In [ ]:
# assign to variable "lens"
lens <- df_2016%>%
    select(ssn, county, tanf_spell_months) %>%
    mutate(rel_length = ifelse(tanf_spell_months > percentile$'.9', 'long', 'not long'))

Unfortunately, there is not a set function within the `tidyverse` or `base R` to calculate proportions. We still can find proportions, but it just requires us to find the count per group first.

Instead of finding the count using the `count()` function as we have in the past, we can find the count using a combination of `group_by()` and `summarize()` here. We will do so because we will need to further manipulate the data frame by each group when we find the proportion of "long" values by county.

> If you do not `filter()` for just 'long' spells in the code below, you will have two rows per county, one of the proportion of 'long' spells, and the other the proportion of 'not long' spells.

In [ ]:
# find count and proportion of "long" by county sorted by highest proportion
lens %>%
    group_by(county, rel_length) %>%
    summarize(n=n()) %>%
    mutate(Proportion = n/sum(n)) %>%
    ungroup() %>%
    filter(rel_length == 'long') %>%
    arrange(desc(Proportion)) %>%
    # we don't need to see rel_length column
    select(-rel_length) %>%
    head()

Finally, instead of just looking at the counties with the highest proportion of longer TANF spells, we can find the `summary()` of the proportions amongst all counties in Indiana.

In [ ]:
# assign to props
props <- lens %>%
    group_by(county, rel_length) %>%
    summarize(n = n()) %>%
    mutate(Proportion = n/sum(n)) %>%
    ungroup() %>%
    filter(rel_length == 'long') %>%
    select(-rel_length)

# see distribution by county
summary(props$Proportion)

<font color=red><h3> Checkpoint 4: Recreate for 2009Q1 </h3></font> 

Find the answers to the questions at the beginning of this section "The TANF Experience" for the 2009Q1 cohort. 

Do the calculations vary significantly from the ones for our 2016Q4 cohort? Are you finding that those from similiar counties in our 2016Q4 cohort are more likely to have longer TANF spells as those in the 2009Q1 cohort?

In [ ]:
# How many individuals are in each cohort?


In [ ]:
# How does the number of individuals in the two cohorts vary by county? 


In [ ]:
# What is the distribution of spell lengths within in our cohort?


In [ ]:
# What are the spell lengths at the 10th, 25th, 50th, 75th and 90th percentiles?


In [ ]:
# Are we seeing a concentration of lengthy spells in specific regions?


## Employment Outcomes of TANF Exiters

Now that we have a better grasp of how the TANF experience can be presented through data, we will take a look at post-exit employment outcomes up to one year after exit. Thus, for our 2016Q4 cohort, we will analyze their employment outcomes for 2017Q1-2017Q4. 

Are members of our cohort finding some sort of sustainable employment where they will not apply for TANF support in the future? Are they finding any employment at all? If so, what are their wages?

To find the answers to these questions, we will first match our two TANF leaver cohorts to the Indiana Unemployment Insurance wage records.

### Match Cohort to UI Wage Records

First, let's take a look at one of Indiana's UI wage records tables, `wage_by_employer`, which is located in the `in_dwd` schema.

> There is another set of tables, `wagesums`, which aggregate earnings for each individual by quarter in Indiana.

In [ ]:
# see Indiana's wage_by_employer table
qry = "
select *
from in_dwd.wage_by_employer
limit 5
"
dbGetQuery(con, qry)

We can see that the `wage_by_employer` table contains wage entries by quarter, employer, and individual, which is perfect for joining it to our cohort. First, we need to create our cohort table in SQL. Luckily, though, just so you all do not have to create temporary tables at the same time, we have already created the 2016 cohort for you in the `ada_tdc_2020` schema using the code below.

> When you create temporary or permanent tables in a relational database using an R Kernel, you will need to run `dbExecute()` instead of `dbGetQuery()` since there is no output to be returned when creating tables. `dbExecute()` will create the table as well as provide the number of rows of the table.

    create table ada_tdc_2020.cohort_2016 as 
    SELECT distinct on (a.ssn)
    a.ssn, a.caseid, a.month, a.tanf_start, a.tanf_end, a.tanf_spell_months, a.tanf_total_months,b.county,
    substring(a.month,1,4) as rep_year, substring(a.month,5,2) as rep_month, extract(year from a.dob) as dob_yr
    FROM in_fssa.person_month a
    INNER JOIN in_fssa.case_month b on a.caseid = b.caseid
    WHERE 
    a.affil = '1' and
    a.tanf_end = TRUE and 
    ssn not in REDACTED and
    substring(a.month,1,4) = '2016' and 
    substring(a.month,5,2) in ('10','11','12')
    order by a.ssn, a.month desc;

In [ ]:
# see cohort_2016 table
qry = "
select *
from ada_tdc_2020.cohort_2016
limit 5
"
dbGetQuery(con, qry)

From here we can join our cohort to the wage table in SQL to obtain earnings for four quarters after exiting the TANF program. We will need to perform a `LEFT JOIN` from `cohort_2016` onto `wage_by_employer`, since we only want wage records for those in our cohort, and we will be able to do so through a common `ssn` across the two tables. We will also select the following information:
- employer (`uiacct`)
- earnings (`wages`)
- industry of their employer (`naics_3_digit`)
- county of their employment (`cnty`)
- first date of quarter (`job_yr_q`)

Let's test some code to see if it appears that our join works as intended.

> The `format()` function in SQL will allow us to convert `year` and `quarter` combinations into dates. We are listing their job date as the first day of those quarters (i.e. Q2 corresponds to April 1). We won't be using this column in this notebook, but will leverage it in a later one.

In [ ]:
# link cohort to ui wage records to get employment outcomes
qry = "
select a.ssn, a.tanf_spell_months, a.tanf_total_months, a.county,
b.year, b.quarter, b.uiacct, b.wages, b.naics_3_digit, b.cnty, format('%s-%s-1', b.year, b.quarter*3-2)::date as job_yr_q
from ada_tdc_2020.cohort_2016 a
left join in_dwd.wage_by_employer b
on a.ssn = b.ssn
where b.year = 2017
limit 5
"
dbGetQuery(con, qry)

In [ ]:
# link cohort to ui wage records to get employment outcomes
qry = "
select a.ssn, a.tanf_spell_months, a.tanf_total_months, a.county,
b.year, b.quarter, b.uiacct, b.wages, b.naics_3_digit, b.cnty, format('%s-%s-1', b.year, b.quarter*3-2)::date as job_yr_q
from ada_tdc_2020.cohort_2016 a
left join in_dwd.wage_by_employer b
on a.ssn = b.ssn
where b.year = 2017
limit 5
"
dbGetQuery(con, qry)

Since the code above works, we used it to create a permanent table containing wage outcomes within one year of exit for our cohort. For your viewing pleasure, we included the code below.

    create table ada_tdc_2020.cohort_2016_earnings as
    select a.ssn, a.tanf_spell_months, a.tanf_total_months, a.county,
    b.year, b.quarter, b.uiacct, b.wages, b.naics_3_digit, b.cnty, 
    format('%s-%s-1', b.year, b.quarter*3-2)::date as job_yr_q
    from ada_tdc_2020.cohort_2016 a
    left join in_dwd.wage_by_employer b
    on a.ssn = b.ssn
    where b.year = 2017

In [ ]:
# see cohort_2016_earnings
qry = "
select *
from ada_tdc_2020.cohort_2016_earnings
limit 5
"
dbGetQuery(con, qry)

<font color=red><h3> Checkpoint 5: Recreate for 2009Q1 </h3></font> 

Select five rows after joining the 2009Q1 cohort to the UI wage records up to a year after exit by joining `ada_tdc_2020.cohort_2009` to `in_dwd.wage_by_employer`. What did you have to change from the code used above to select 5 rows?

In [ ]:
# select five rows from ada_tdc_2020.cohort_2009_wages by joining ada_tdc_2020.cohort_2009 to in_dwd.wage_by_employer



## Post-TANF Employment Outcomes

Now, using our new table, `ada_tdc_2020.cohort_2016_earnings`, we will try to get a better sense of where TANF recipients are finding employment and their subsequent earnings by answering specific questions:

1. How many leavers found employment in at least one quarter the following year after exit? What percentage is this of our original cohort?
1. What were their annualized earnings? What about their average earnings per quarter?
1. What were the most popular industries of employment? Do average quarterly earnings per person vary significantly amongst these industries?
1. How many different employers did they have in this time frame? In how many quarters were they employed? In how many different counties were they employed?

<font color=green><h4>Guiding Question 1: How many leavers found employment in at least one quarter the following year after exit? What percentage is this of our original cohort?</h4></font> 

First, let's read our `ada_tdc_2020.cohort_2016_earnings` table into R as `df_2016_wages`.

In [ ]:
# read table into R
qry = "
select *
from ada_tdc_2020.cohort_2016_earnings
"
df_2016_wages = dbGetQuery(con, qry)

To find the amount of leavers that found employment in at least one quarter over the course of this time frame, we can simply count the number of unique `ssn` values. The `tidyverse's` `n_distinct()` function makes that really easy for us.

In [ ]:
# number of leavers who found employment in at least one quarter 
df_2016_wages %>%
    summarize(n = n_distinct(ssn))

Then, to find the percentage of our total cohort, we can leverage `df_2016`, since it contains information on our entire cohort, as well as introduce another function, `percent`, that outputs a percentage as a character vector.

In [ ]:
# percentage of our cohort that was employed during at least one quarter the following year after exit
percent(n_distinct(df_2016_wages$ssn) / n_distinct(df_2016$ssn), .01)

Are you suprised by this percentage? How do you think it will differ for the 2009Q1 cohort?

<font color=green><h4>Guiding Question 2: What were their annualized earnings? What about their average earnings per quarter?</h4></font> 

To calculate annualized earnings for each `ssn` across all jobs, we will simply add all of their earnings within this year. 

> Because (as you will see) a large portion of our cohort wasn't employed all four quarters, we will also look at the average earnings per quarter.

In [ ]:
# see if annualized earnings were calculated properly
df_2016_wages %>%
    group_by(ssn) %>%
    summarize(total_wages = sum(wages)) %>%
    head()

In [ ]:
# save annualized earnings as wages_one_year
wages_one_year <- df_2016_wages %>%
    group_by(ssn) %>%
    summarize(total_wages = sum(wages)) %>%
    ungroup()

Now, we can find a numerical summary of these annualized earnings using both `summary()` and `summarize()` combined with `quantile()`.

In [ ]:
# quick numerical distribution of total_wages
summary(wages_one_year$total_wages)

In [ ]:
# more nuanced look at annual wage distribution
wages_one_year %>%
    summarize('.1' = quantile(total_wages, .1),
              '.25' = quantile(total_wages, .25),
              '.5' = quantile(total_wages, .5),
              '.75' = quantile(total_wages, .75),
              '.9' = quantile(total_wages, .9)
             )

At this point, you might be wondering why you haven't seen a visual representation of these earnings. Don't worry, you will get to data visualization in the next [notebook](04_01_Data_Visualization.ipynb).

Let's take a look at the quarterly wage distribution by adding `quarter` as an argument to our `group_by()` statement to see if we can learn more about our cohort's wage distribution.

In [ ]:
# see if we can find quarterly earnings
df_2016_wages %>%
    group_by(ssn, quarter) %>%
    summarize(quarterly_wages = sum(wages)) %>%
    ungroup() %>%
    head()

In [ ]:
# save quarterly earnings as wages_quarter
wages_quarter <- df_2016_wages %>%
    group_by(ssn, quarter) %>%
    summarize(quarterly_wages = sum(wages)) %>%
    ungroup()

In [ ]:
# quick numerical distribution of quarterly_wages
summary(wages_quarter$quarterly_wages)

In [ ]:
# more nuanced look at quarterly earnings distribution
wages_quarter %>%
    summarize('.1' = quantile(quarterly_wages, .1),
              '.25' = quantile(quarterly_wages, .25),
              '.5' = quantile(quarterly_wages, .5),
              '.75' = quantile(quarterly_wages, .75),
              '.9' = quantile(quarterly_wages, .9)
             )

Do you think looking at quarterly or annual wage distributions tells you more about this cohort's employment outcomes? Why? Is there anything else you would like to know about this cohort before answering this question?

<font color=green><h4>Guiding Question 3: What were the most popular industries of employment? Do average quarterly earnings per person vary significantly amongst these industries?</h4></font> 

For this question, we will focus on the 10 most common industries. At this point, you may have started to notice some similar patterns in `tidyverse` chains of commands in discovering certain attributes about our population in question.

> The second question infers that we will sum each individual's earnings by quarter within a specific industry. Therefore, if someone worked two jobs in the same industry in the same quarter, the earnings from these two jobs would be combined for the quarterly representation of quarterly earnings for this individual in this industry.

In [ ]:
# see if we can find 10 most common naics
df_2016_wages %>%
    group_by(naics_3_digit) %>%
    summarize(num = n_distinct(ssn)) %>% 
    arrange(desc(num)) %>%
    head(10)

In [ ]:
# save 10 most common naics as pop_naics
pop_naics = df_2016_wages %>%
    group_by(naics_3_digit) %>%
    summarize(num = n_distinct(ssn)) %>% 
    arrange(desc(num)) %>%
    head(10)

Although finding these 3-digit NAICS codes is great, it would be far more helpful if we knew the industries corresponding to these NAICS codes. Luckily, we have a table `naics_2017` in the `public` schema that can function as a crosswalk for us.

In [ ]:
# see the naics_2017 table
qry = '
select *
from public.naics_2017
limit 5
'
dbGetQuery(con, qry)

In [ ]:
# read naics_2017 table into R as naics
qry = '
select *
from public.naics_2017
'
naics = dbGetQuery(con, qry)

Similar to how we performed a `LEFT JOIN` to match our cohort to the UI wage records, we can follow a similar process here. Instead of using SQL, though, we will use the `tidyverse's` `left_join` function. 
> Instead of using `on` like in SQL, you need to supply an argument to `by` to designate the columns you would like to join on.

In [ ]:
# get industry names of most popular naics
pop_naics %>% 
    left_join(naics, by=c('naics_3_digit' = 'naics_us_code')) %>%
    # don't include the other columns
    select(-c(seq_no,naics_3_digit)) %>%
    # sort order of columns
    select(naics_us_title, num)

Now, to find the average earnings of members of our cohort employed in these industries, we can subset `df_2016_wages` using the `tidyverse's` `filter()` function to just include wage records for those employed in the 10 most popular industries.

> `%in%` works in the same fashion as SQL's `in`. 

In [ ]:
# make sure we can get ssn, earnings, naics and quarter for all wage records in our 10 most popular naics codes
df_2016_wages %>%
    filter(naics_3_digit %in% pop_naics$naics_3_digit) %>%
    select(ssn, wages, naics_3_digit, quarter) %>%
    head()

In [ ]:
# assign wage records for our 2016Q4 cohort for 10 most popular naics to wages_pop_naics
wages_pop_naics <- df_2016_wages %>%
    filter(naics_3_digit %in% pop_naics$naics_3_digit) %>%
    select(ssn, wages, naics_3_digit, quarter)

In [ ]:
# make sure we are finding quarterly earnings for each ssn, quarter, naics code combination
wages_pop_naics %>%
    group_by(ssn, quarter, naics_3_digit) %>%
    summarize(tot_wages = sum(wages)) %>%
    ungroup() %>%
    head()

In [ ]:
# save to quarterly_naics
quarterly_naics <- wages_pop_naics %>%
    group_by(ssn, quarter, naics_3_digit) %>%
    summarize(tot_wages = sum(wages)) %>%
    ungroup()

Now, we can group `quarterly_naics` by `naics_3_digit` to find average quarterly earnings per `ssn`.

In [ ]:
# find average quarterly earnings by industry and include number of people employed at least one quarter in each industry
quarterly_naics %>%
    group_by(naics_3_digit) %>%
    summarize(avg_wages = mean(tot_wages),
             num_ssns = n_distinct(ssn)) %>%
    arrange(desc(num_ssns))

To add the cherry on top, we can add in the industry names corresponding to each NAICS code again.

In [ ]:
# save results from above to pop_naics_wages
pop_naics_wages <- quarterly_naics %>%
    group_by(naics_3_digit) %>%
    summarize(avg_wages = mean(tot_wages),
             num_ssns = n_distinct(ssn)) %>%
    arrange(desc(num_ssns))

In [ ]:
# add in industry names
pop_naics_wages %>% 
    left_join(naics, by=c('naics_3_digit' = 'naics_us_code')) %>%
    # don't include the other columns
    select(-c(seq_no,naics_3_digit)) %>%
    # switch order of columns
    select(naics_us_title, avg_wages, num_ssns)

Are you suprised by these results at all?

<font color=green><h4>Guiding Question 4: How many different employers did they have in this time frame? In how many quarters were they employed? In how many different counties were they employed?</h4></font> 

Yes, answering three questions in a tiny section may seem daunting. However, we can answer these three questions from the same data frame. To create that data frame, we will aggregate the number of employers, quarters of employment, and counties by each `ssn` in `df_2016_wages` using `n_distinct()`.

In [ ]:
# make sure we can find amount of diff employers, amount of quarters worked and amount of counties by ssn
df_2016_wages %>%
    group_by(ssn) %>%
    summarize(num_employers = n_distinct(uiacct),
              quarters_work = n_distinct(quarter),
              counties_work = n_distinct(cnty)
    ) %>%
    ungroup() %>%
    head()

In [ ]:
# save as "stats"
stats <- df_2016_wages %>%
    group_by(ssn) %>%
    summarize(num_employers = n_distinct(uiacct),
              quarters_work = n_distinct(quarter),
              counties_work = n_distinct(cnty)
    ) %>%
    ungroup()

Now that we have created our base data frame to answer this set of questions, we can separately group by `num_employers`, `quarters_work`, and `counties_work`.

In [ ]:
# see number of ssns by amount of employers
stats %>%
    group_by(num_employers) %>%
    summarize(n=n())

Are you suprised by this result?

In [ ]:
# number of quarters worked per ssn
stats %>%
    group_by(quarters_work) %>%
    summarize(n=n())

Here's a better look of why it may be more accurate to report quarterly earnings as opposed to annual ones (not everyone was working all four quarters).

In [ ]:
# number of total counties worked in
stats %>%
    group_by(counties_work) %>%
    summarize(n=n())

As an aside, while we are looking at the amount of counties members of our cohort worked in during this time frame, we can also find the most popular counties of work in Indiana from `df_2016_wages`.

One thing to note from the data dictionary on the UI wage records: There are missing county codes--these pertain to entries with values `999`, `995`, or `900`. We'll filter these ones from our analysis here.

In [ ]:
# most popular counties
pop_cntys <- df_2016_wages %>%
    filter(!(cnty %in% c(900, 955, 999))) %>%
    group_by(cnty) %>%
    summarize(n = n_distinct(ssn)) %>%
    arrange(desc(n)) %>%
    head(10)

pop_cntys %>%
    left_join(counties, c("cnty"="county")) %>%
    select(name, n)

How does this compare to the most popular counties to receive TANF assistance for our cohort?

<font color=red><h3> Checkpoint 6: Recreate for 2009Q1 </h3></font> 

Find the answers to the questions at the beginning of this section "Post-TANF Employment Outcomes" for the 2009Q1 cohort. 

Do the calculations vary significantly from the ones for our 2016Q4 cohort? Did a larger percentage of our 2009Q1 cohort appear in the UI wage records? How can you compare and contrast calculations for the other questions?

In [ ]:
# How many leavers found employment in at least one quarter the following year after exit? 
# What percentage is this of our original cohort?



In [ ]:
# What were their annualized earnings? 
# What about their average earnings per quarter?



In [ ]:
# What were the most popular industries of employment?
# Do average quarterly earnings per person vary significantly amongst these industries?


In [ ]:
# How many different employers did they have in this time frame? 
# In how many quarters were they employed? 
# In how many different counties were they employed?
# What were the most popular counties of employment?

In this notebook, you have covered how to identify the cohort that you are interested in from a database and save it as data frame in R. You have also seen how to conduct descriptive analyses in R, such as checking missing values and breaking down the sample based on the variables that you are interested.

After you find interesting results, you may want to present them in the form of pictures, or visualizations. In the next notebook, which will cover [Data Visualization](Data_Visualization.ipynb), we will show you how to leverage more of the `tidyverse` suite of packages in R to display some of your findings.